In [ ]:
import zipfile
import os
import pandas as pd
import re
from datetime import datetime
import matplotlib.pyplot as plt


# Leitura da base
with open("/content/drive/MyDrive/Whatsapp_NLP/_chat.txt", 'r', encoding='utf-8') as file:
    chat_data = file.readlines()


# Função para analisar cada linha do arquivo de chat
def parse_line(line):
    # Expressão regular para corresponder à data e hora
    match = re.match(r'\[(\d{2}/\d{2}/\d{2}), (\d{2}:\d{2}:\d{2})\] (.*?): (.*)', line)
    if match:
        date, time, user, message = match.groups()
        datetime_str = date + ' ' + time
        datetime_obj = datetime.strptime(datetime_str, '%d/%m/%y %H:%M:%S')
        return datetime_obj, user, message
    else:
        return None, None, line

# Analise os dados do chat
parsed_data = [parse_line(line) for line in chat_data]

# Crie um DataFrame
chat_df = pd.DataFrame(parsed_data, columns=['datetime', 'user', 'message'])

# Exiba as primeiras linhas do DataFrame
chat_df.head()

# Reamostre os dados para obter o número de mensagens por dia
messages_per_day = chat_df.resample('D', on='datetime').size()

max(messages_per_day)

# Plote o número de mensagens por dia
plt.figure(figsize=(12, 6))
messages_per_day.plot()
plt.title('Número de Mensagens por Dia')
plt.xlabel('Data')
plt.ylabel('Número de Mensagens')
plt.grid(True)
plt.show()

# Conte o número de mensagens enviadas por cada usuário
user_message_counts = chat_df['user'].value_counts()

# Plote o número de mensagens enviadas por cada usuário
plt.figure(figsize=(12, 6))
user_message_counts.plot(kind='bar')
plt.title('Número de Mensagens por Usuário')
plt.xlabel('Usuário')
plt.ylabel('Número de Mensagens')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Categorize as mensagens
def categorize_message(message):
    if 'sticker omitted' in message:
        return 'Sticker'
    elif 'image omitted' in message:
        return 'Imagem'
    elif 'GIF omitted' in message:
        return 'GIF'
    elif 'video omitted' in message:
        return 'Vídeo'
    elif 'audio omitted' in message:
        return 'Áudio'
    elif 'document omitted' in message:
        return 'Documento'
    else:
        return 'Texto'

# Aplique a função para categorizar cada mensagem
chat_df['message_type'] = chat_df['message'].apply(categorize_message)

# Conte o número de cada tipo de mensagem
message_type_counts = chat_df['message_type'].value_counts()

# Plote a distribuição dos tipos de mensagens
plt.figure(figsize=(12, 6))
message_type_counts.plot(kind='bar')
plt.title('Distribuição dos Tipos de Mensagens')
plt.xlabel('Tipo de Mensagem')
plt.ylabel('Número de Mensagens')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

import pandas as pd
import re
from datetime import datetime, timedelta
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
# import ace_tools as tools
import nltk
from nltk.corpus import stopwords
from collections import Counter
import spacy
import zipfile
import os
import pandas as pd
import re
from datetime import datetime
import matplotlib.pyplot as plt

# Baixar as stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))
additional_stopwords = {'kkkk', 'kkkkk', 'kkkkkk', 'kkkkkkk', 'kkkkkkkk', 'kkkkkkkkk', 'kkkkkkkkkk',
                        'vai', 'vou', 'cara', 'vou', 'eae', 'eai', 'vc','pra','aí','tá','lá', 'nao',
                        'q','então','pq','n','ta', 'sim', 'aqui', 'pro', 'mto', 'eh', '@5561996886412',
                        '@5561998118003','@5561998660927','@5561999011118', '@5561998669648','@5561983404441',
                        '@5561998172241','@5561981154414','@5561991074343', 'acho', 'boa', 'qm', 'tbm'}
stop_words.update(additional_stopwords)


# Leitura da base
with open("/content/drive/MyDrive/Whatsapp_NLP/_chat.txt", 'r', encoding='utf-8') as file:
    chat_data = file.readlines()


# Função para analisar cada linha do arquivo de chat
def parse_line(line):
    match = re.match(r'\[(\d{2}/\d{2}/\d{2}), (\d{2}:\d{2}:\d{2})\] (.*?): (.*)', line)
    if match:
        date, time, user, message = match.groups()
        datetime_str = date + ' ' + time
        datetime_obj = datetime.strptime(datetime_str, '%d/%m/%y %H:%M:%S')
        return datetime_obj, message
    else:
        return None, None

# Analise os dados do chat
parsed_data = [parse_line(line) for line in chat_data]
parsed_data = [data for data in parsed_data if data[0] is not None]

# Crie um DataFrame
chat_df = pd.DataFrame(parsed_data, columns=['datetime', 'message'])

# Remover mensagens que contêm "omitted"
chat_df = chat_df[~chat_df['message'].str.contains('omitted', na=False)]
chat_df = chat_df[~chat_df['message'].str.contains('This message was deleted', na=False)]

# Converter a coluna datetime para objeto datetime
chat_df['datetime'] = pd.to_datetime(chat_df['datetime'])

# Ordenar o DataFrame por datetime
chat_df = chat_df.sort_values(by='datetime')

# Calcular a diferença de tempo entre as mensagens
chat_df['time_diff'] = chat_df['datetime'].diff()

# Definir um novo grupo se a diferença de tempo for superior a duas horas
chat_df['new_group'] = chat_df['time_diff'] > pd.Timedelta(hours=2)

# Criar um identificador de grupo cumulativo
chat_df['group_id'] = chat_df['new_group'].cumsum()

# Concatenar mensagens dentro de cada grupo
grouped_df = chat_df.groupby('group_id').agg({
    'datetime': 'first',  # Manter a primeira data e hora do grupo
    'message': ' '.join  # Concatenar mensagens
}).reset_index()

# Preprocessar o texto e remover stopwords
def preprocess_text(text):
    text = re.sub(r'\W', ' ', str(text))  # Remove caracteres não alfanuméricos
    text = re.sub(r'\s+', ' ', text, flags=re.I)  # Remove múltiplos espaços
    text = text.lower()  # Converte para minúsculas
    words = text.split()
    words = [word for word in words if word not in stop_words and not word.startswith('5561') and not word.startswith('https')]  # Remove stopwords
    return ' '.join(words)

grouped_df['cleaned_message'] = grouped_df['message'].apply(preprocess_text)

# Filtrar grupos com mais de 100 palavras
grouped_df['word_count'] = grouped_df['cleaned_message'].apply(lambda x: len(x.split()))
filtered_df = grouped_df[grouped_df['word_count'] > 50]

# Concatenar todas as mensagens para análise de frequência
all_words = ' '.join(filtered_df['cleaned_message']).split()

# Contar a frequência das palavras
word_freq = Counter(all_words)

# Converter para DataFrame para visualização
word_freq_df = pd.DataFrame(word_freq.items(), columns=['word', 'frequency']).sort_values(by='frequency', ascending=False)

# Mostrar as 20 palavras mais frequentes
top_words = word_freq_df.head(20)
print(top_words)

# Plotar as 20 palavras mais frequentes
plt.figure(figsize=(10, 6))
plt.bar(top_words['word'], top_words['frequency'], color='purple')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('Top 20 Most Frequent Words')
plt.xticks(rotation=45)
plt.show()

# Verificar o número de grupos após o filtro
print(f"Número de grupos com mais de 100 palavras: {filtered_df.shape[0]}")
print(f"Número de grupos totais: {grouped_df.shape[0]}")

# Salvar o DataFrame filtrado para a próxima etapa
filtered_df.to_csv('filtered_grouped_chat.csv', index=False)

import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP

# Carregar o DataFrame filtrado
filtered_df = pd.read_csv('/content/filtered_grouped_chat.csv')

# Utilizar modelo de linguagem pré-treinado para gerar embeddings
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(filtered_df['cleaned_message'], show_progress_bar=True)

import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns
n_top = 15

# Reduzir a dimensionalidade dos embeddings com UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine')
umap_embeddings = umap_model.fit_transform(embeddings)

# Ajustar parâmetros do BERTopic
topic_model = BERTopic(
    nr_topics=n_top,
    language="portuguese",
    min_topic_size=5,
    n_gram_range=(1, 3),
    calculate_probabilities=True
)

topics, probabilities = topic_model.fit_transform(filtered_df['cleaned_message'], umap_embeddings)

# Adicionar a classificação dos tópicos ao DataFrame
filtered_df['topic'] = topics

# Visualizar os tópicos
topic_info = topic_model.get_topic_info()
print(f"{n_top} Tópicos mais discutidos no grupo")
print(topic_info)

# Visualizar as palavras-chave de cada tópico
for topic in range(len(topic_model.get_topics())):
    if topic == -1:  # Outliers
        continue
    print(f"Tópico {topic}:")
    print(topic_model.get_topic(topic))

# Obter embeddings dos tópicos
topic_embeddings = topic_model.topic_embeddings_

# Agrupar tópicos utilizando clustering hierárquico
clustering_model = AgglomerativeClustering(n_clusters=5)
topic_labels = clustering_model.fit_predict(topic_embeddings)

# Adicionar os rótulos de cluster aos tópicos
topic_info['cluster'] = topic_labels

# Visualizar os clusters de tópicos
print("Clusters dos topicos")
print(topic_info)

# Visualizar os tópicos em um gráfico interativo (opcional)
topic_model.visualize_topics()

# Visualizar a distribuição dos tópicos ao longo das mensagens (opcional)
topic_model.visualize_distribution(probabilities[0])

# Salvar o DataFrame resultante em um novo CSV
filtered_df.to_csv('filtered_grouped_chat_with_topics.csv', index=False)
topic_info.to_csv('topic_info_with_clusters.csv', index=False)

# Exibir o DataFrame resultante
# print(filtered_df.head(10))  # Exibir as primeiras 10 linhas do DataFrame
# print(topic_info.head(10))  # Exibir as primeiras 10 linhas do DataFrame de tópicos

#for i in range(n_top):
    #filtered_df[filtered_df['topic'] == i].to_csv(f'filtered_grouped_chat_with_topics_{i+1}.csv', index=False)

# Exibir o DataFrame resultante
# print(filtered_df.head(10))  # Exibir as primeiras 10 linhas do DataFrame
